# Scraping Twitter

In [ ]:
from googletrans import Translator

import pandas as pd
import time
import traceback
import tweepy

In [ ]:
#the values below are fake due to security reason. Substitute yours unique credentials.
consumer_key = "1q1q1q1q1q1q1q1q1q1q1q1q1q"
consumer_secret = "2w2w2w2w2w2w2w2w2w2w2w2w2w2w2w2w"
access_token = "12121212121221-4r4r4r4r4r4r4r4rr4"
access_token_secret = "5t5t5t5t5t5t5t5t5t5t5t5t5t5t5t5"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

The code below let you to scrape tweets from specific tweeter users. 

I have chosen to scrape 200 most recent tweets from three news media accounts. Scraped data I save in csv file.

In [ ]:
tweets, news_media = [], ['cnnbrk', 'BBCWorld', 'nytimes']

count = 200

try: 
    for news in news_media:
        print('Scraping: ', news)
        for tweet in api.user_timeline(id = news, count=count):
            tweets.append((news, tweet.created_at, tweet.id, tweet.text))
        time.sleep(15)    
    df = pd.DataFrame(tweets, columns = ['news_media', 'created_at', 'tweet_id', 'text'])
    df.to_csv('news_tweets.csv')
    print("Completed.")
    
except BaseException as e:
    traceback.print_exc()

The code below lets you to scrape tweets by keywords. You are getting most tweet with the keywords. Additionally, if the tweet was not in English, I would translate them into English. Save all data in csv.

Therefore, I extract for every tweet: 

* time it was created_at
* tweet_id 
* tweet text 
* username (unique tweeter username) 
* name (name written by user)
* location (where the user is from)
* followers_count (number of user's followers)
* original language

In [ ]:
tweets, text_query = [], 'coronavirus OR virus OR covid-19 OR covid19 OR pandemic'
count = 100

try:
    for tweet in api.search(q=text_query, count=count, result_type='recent',
                           include_entities=True,
                           monitor_rate_limit=True, 
                           wait_on_rate_limit=True):
        tweet_text = ''
            
        try:
            tr = Translator()
            
            if len(str(tweet.lang)) > 0 and tweet.lang not in ['en', 'und']:
                tweet_text = tr.translate(tweet.text).text
            else:
                tweet_text = tweet.text
                
        except Exception as e:
            traceback.print_exc()
            pass
        
        tweets.append((tweet.created_at, tweet.id, tweet_text, tweet.user.screen_name, tweet.user.name, 
                               tweet.user.location, tweet.user.followers_count, tweet.lang))
        
        df = pd.DataFrame(tweets, columns = ['created_at', 'tweet_id', 'text', 'username', 'name', 'location', 'followers_count', 'lang'])
        df.to_csv('tweets_by_keywords.csv')
        time.sleep(3)
        
    print("Completed.")
    
except BaseException as e:
    traceback.print_exc()        